# Advent of Code 2023: Day 22
https://adventofcode.com/2023/day/22


## Part 1
Count the number of 3d blocks that can be removed without causing another to fall

### Get the data into a list of lists of strings

In [1]:
myfile = open('input.txt', 'r')
data = myfile.read()
data_list = data.split('\n')
data_list = [[list(map(int, s.split(','))) for s in line.split('~')]
             for line in data_list]
data_list = sorted(data_list, key=lambda l: l[0][2])

### Create function to drop each block as far as possible
Return a dictionary which tracks which blocks (value) a single block (key) has on top of it.

In [2]:
def dropBlocks(data):
  on_top = {}
  z = {}
  for i, block in enumerate(data):
    while block[0][2] > 0:
      if block[0][2] == 1:
        on_top[str(block)] = set()
        for j in range(1, block[1][2]+1):
          if z.get(j):
            z[j].append(block)
          else:
            z[j] = [block]
        break
      else:
        x = range(block[0][0],block[1][0]+1)
        y = range(block[0][1],block[1][1]+1)
        level = z.get(block[0][2]-1)
        if level:
          empty = True
          for j in x:
            for l in y:
              for cell in level:
                if (j >= cell[0][0] and j <= cell[1][0]) and (l >= cell[0][1] and l <= cell[1][1]):
                  on_top[str(cell)].add(str(block))
                  empty = False
          if empty:
            block[0][2] -= 1
            block[1][2] -= 1
          else:
            on_top[str(block)] = set()
            for j in range(block[0][2], block[1][2]+1):
              if z.get(j):
                z[j].append(block)
              else:
                z[j] = [block]
            break
        else:
          block[0][2] -= 1
          block[1][2] -= 1
  return on_top

### Create function to check how many blocks can be removed, individually, without causing any other block to fall

Also return the _below_ dictionary, which is an inversion of the input dictionary, for future use.

In [3]:
def checkRemovable(on_dict):
  below = {}
  d = set()
  singles = []
  for key, value in on_dict.items():
    if not value:
      d.add(key)
    else:
      for v in value:
        if below.get(v):
          below[v].add(key)
        else:
          below[v] = set([key])
  for key, value in below.items():
    if len(value) == 1:
      singles.append(list(value)[0])
  for key, value in below.items():
    if len(value) > 1:
      for v in value:
        if v not in singles:
          d.add(v)
  return len(d), below

### Count removable blocks

In [4]:
on_top = dropBlocks(data_list)
removable, below = checkRemovable(on_top)
removable

488

## Part 2
For each block, check how many blocks would fall if it were removed

### Create function to calculate how many blocks would fall if a given block is removed

In [5]:
def countFalls(block, on_dict, under_dict):
  fallen = set()
  if not on_dict[block]:
    return len(fallen)
  else:
    fallen = set([block])
    fq = set()
    fq.update(on_dict[block])
    while fq:
      cur = fq.pop()
      if len(under_dict[cur]) == 1:
        fallen.add(cur)
        fq.update(on_dict[cur])
      elif under_dict[cur].issubset(fallen):
        fallen.add(cur)
        fq.update(on_dict[cur])
  return len(fallen)-1

### Sum up how many blocks would fall if each individual block was removed

In [6]:
total = 0
for k in on_top.keys():
  total += countFalls(k, on_top, below)
total

79465